# v2 DB Migration Script

Run this only after creating a fresh db with the schema already migrated.

In [176]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
from uuid_extensions import uuid7str

In [177]:
load_dotenv()

CSV_DIRECTORY = "./csv"
DB_CONNECTION_STRING = os.environ.get("DB_CONNECTION_STRING")

# Load csv files
user_df = pd.read_csv(
    f"{CSV_DIRECTORY}/cfc-website_user.csv", dtype={"student_number": str}
)
payment_df = pd.read_csv(f"{CSV_DIRECTORY}/cfc-website_payment.csv", dtype={"amount": float})

engine = create_engine(DB_CONNECTION_STRING)

In [178]:
# ETL for user
user_table = user_df.drop(columns=["xata.createdAt", "xata.updatedAt", "xata.version"])
user_table["clerk_id"] = user_table["id"]
user_table["id"] = user_table["id"].apply(lambda _: uuid7str())

In [179]:
# Check all payment.user_id maps to a user.clerk_id
assert payment_df["user_id"].isin(user_table["clerk_id"]).all()

# ETL for payment
payment_table = payment_df.drop(
    columns=["xata.createdAt", "xata.updatedAt", "xata.version"]
)

payment_table["id"] = payment_table["id"].apply(lambda _: uuid7str())

# Map payment.user_id to user.id
payment_table["user_id"] = payment_table["user_id"].map(
    user_table.set_index("clerk_id")["id"]
)

In [175]:
# Insert data to db

with engine.connect() as conn:
    user_table.to_sql("cfc-website_user", conn, if_exists="append", index=False)
    payment_table.to_sql("cfc-website_payment", conn, if_exists="append", index=False)